# Load Data

In [1]:
import re
import pandas as pd

# Read the dialogue from the file
file_path = "data/Whatsapp_Chat.txt"
with open(file_path, "r", encoding="utf-8") as file:
    dialogue = file.read()

lines = dialogue.strip().split('\n')
parsed_data = []
current_date = ""
hanif_message = ""
cuty_pie_message = ""

for line in lines:
    match = re.match(r'(\d{2}/\d{2}/\d{4}, \d{2}:\d{2}) - (.+?): (.+)', line)
    if match:
        date = match.group(1)
        sender = match.group(2)
        message = match.group(3)
        
        if sender == 'Muhammad Hanif Fajari':
            hanif_message = message
        elif sender == '😡 My Cuty Pie😡':
            cuty_pie_message = message
            if current_date == date:
                parsed_data.append((current_date, hanif_message, cuty_pie_message))
        
        current_date = date

df = pd.DataFrame(parsed_data, columns=['date', 'Muhammad Hanif Fajari', '😡 My Cuty Pie😡'])

In [2]:
df =df.rename(columns={'😡 My Cuty Pie😡':'question','Muhammad Hanif Fajari':'answer'})

In [6]:
df.drop('date',axis=1,inplace=True)

In [8]:
df.to_csv('chat_whatsapp.csv')

# Data Preprocessing

In [2]:
import time
import torch
from transformers import Trainer, TrainingArguments
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer,AutoModel
import torch
import time
import evaluate
import pandas as pd
import numpy as np

In [6]:
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
torch.cuda.is_available()

False

In [20]:
df = df.replace('',np.nan).fillna(method='bfill')

In [52]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [53]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
percentage of trainable model parameters: 100.00%


In [33]:
df

,answer,question
0,rajin banget be bawa naskun banyak segitu,aku jadi kangen mereka🤣
1,rajin banget be bawa naskun banyak segitu,iyaa wkwk
2,rajin banget be bawa naskun banyak segitu,haha
3,bawa aja ke cosmax bisa ga si be mereka2 itu wkwk,ga bisa wkwk
4,bawa aja ke cosmax bisa ga si be mereka2 itu wkwk,wkwkwkwk
...,...,...
10998,lagi bikin pipeline codingan kenapa sayangku?,bubi
10999,ini bub,<Media omitted>
11000,abang poulet lagi naik pesawat bub,iyaa wkwk putih bgt
11001,bub mam apa kamu bebi,beli di baharii


# Train Model

## Zero Inference

In [54]:
index = 150

question = df['question'][index]
answer = df['answer'][index]

prompt = f"""
jawab pertanyaan berikut ini.

{question}

jawaban:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{prompt}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{answer}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

---------------------------------------------------------------------------------------------------
INPUT PROMPT:

jawab pertanyaan berikut ini.

udh enek

jawaban:

---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
be di pocin

---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
iii tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun tahun 


In [55]:
prompt = 'apa itu google?'
inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"], 
        max_new_tokens=200,
    )[0], 
    skip_special_tokens=True
)
print(prompt)
print(output)

apa itu google?
Google


## Perform Fine Tuning